<a href="https://colab.research.google.com/github/tedk108/Media-Coverage-of-the-Three-Trump-Supreme-Court-Appointees/blob/main/sa_sc_justice_in_4medias_20201214.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **SC Senate Confirmations in the Media: Sentiment Analysis**
Jon Chun
IPHS200

Analysis:
* https://www.researchgate.net/publication/336450246_Visualizing_narrative_patterns_in_online_news_media/download 

Tech References:
* https://newspaper.readthedocs.io/en/latest/
* https://github.com/jarun/ddgr 
* https://cran.r-project.org/web/packages/syuzhet/index.html 
* https://santhoshse7en.github.io/news-fetch/ (not working)

Strategy:

* create dataset of new articles from the 4 media outlets for each of the 3 SC Nominees
* apply sentiment analysis over these 4 media coverage timelines for each of the 3 SC Nominees 
* analyze and interpret results


Nominees:

* (Nom - Hearings - Vote - Confirmed + 1 mo)
* Gorsuch (31 Jan 2017 - 20-22 Mar 2017 - 7 Apr 2017 - ) 
* Kavanaugh (9 Jul 2018 - 4-7 Sep 2018 Confirmation hearing - 27 Sep 2018 additional hearing on sexual abuse allegations - 6 Oct 2018 + 1mo)
* Coney-Barrett ( 26 Sep 2020 - 12-14 Oct 2020 - 26 Oct 2020 + 1mo)

News Sources:

* CNN
* ABC
* MSNBC
* FoxNew

# **0. Setup and Configuration**

In [ ]:
import pprint

### Install Google Search API for Python

In [ ]:
!pip install google-api-python-client

### Install DuckDuckGo CLI Search package 'ddgr'

In [ ]:
!pip install ddgr

In [ ]:
import ddgr

In [ ]:
!ls

sample_data


In [ ]:
!rm coney-barrett_cnn_202009.json

In [ ]:
%%capture cb_cnn_20200920 --no-stderr

!ddgr -n 25 -w cnn.com --json coney-barrett september 20, 2020

In [ ]:
print(cb_cnn_20200920.stdout)

[
  {
    "abstract": "President Donald Trump intends to choose Amy Coney Barrett to be the new Supreme Court justice, according to multiple senior Republican sources with knowledge Sources: Trump intends to nominate Amy Coney Barrett for Supreme Court. Updated 1533 GMT (2333 HKT) September 26, 2020.",
    "title": "Amy Coney Barrett is Trump's intended choice for Supreme Court...",
    "url": "https://edition.cnn.com/2020/09/25/politics/donald-trump-amy-coney-barrett-supreme-court/index.html"
  },
  {
    "abstract": "Updated 0020 GMT (0820 HKT) September 27, 2020. \"The American people should make no mistake—a vote by any Senator for Judge Amy Coney Barrett is a vote to strike down the Affordable Care Act and eliminate protections for millions of Americans with pre-existing conditions,\" Schumer said.",
    "title": "Democrats say Barrett's nomination is all about the future of...",
    "url": "https://edition.cnn.com/2020/09/26/politics/democrats-biden-amy-coney-barrett-nomination/i

In [ ]:
import json

In [ ]:
json.loads(cb_cnn_20200920.stdout)

[{'abstract': 'President Donald Trump intends to choose Amy Coney Barrett to be the new Supreme Court justice, according to multiple senior Republican sources with knowledge Sources: Trump intends to nominate Amy Coney Barrett for Supreme Court. Updated 1533 GMT (2333 HKT) September 26, 2020.',
  'title': "Amy Coney Barrett is Trump's intended choice for Supreme Court...",
  'url': 'https://edition.cnn.com/2020/09/25/politics/donald-trump-amy-coney-barrett-supreme-court/index.html'},
 {'abstract': 'Updated 0020 GMT (0820 HKT) September 27, 2020. "The American people should make no mistake—a vote by any Senator for Judge Amy Coney Barrett is a vote to strike down the Affordable Care Act and eliminate protections for millions of Americans with pre-existing conditions," Schumer said.',
  'title': "Democrats say Barrett's nomination is all about the future of...",
  'url': 'https://edition.cnn.com/2020/09/26/politics/democrats-biden-amy-coney-barrett-nomination/index.html'},
 {'abstract': 

In [ ]:
cb_cnn_20200920_dt = list(cb_cnn_20200920.stdout)
print(cb_cnn_20200920_dt[200])


e


In [ ]:
!ddgr -n 100 -w cnn.com --json coney-barrett october 2020 >> coney-barrett_cnn_202010.json

In [ ]:
!ddgr -n 100 -w cnn.com --json coney-barrett november 2020 >> coney-barrett_cnn_202011.json

In [ ]:
!ls

coney-barrett_cnn_202009.json  sample_data


In [ ]:
!rm coney*

In [ ]:
!ls

sample_data


In [ ]:
from gsearch.googlesearch import search

In [ ]:
results = search('Full Stack Developer')  # returns 10 or less results

In [ ]:
results

[]

### Install Duck Duck Search Python Package 'python-duckduckgo'

In [ ]:
!git clone https://github.com/mikejs/python-duckduckgo.git

fatal: destination path 'python-duckduckgo' already exists and is not an empty directory.


In [ ]:
!ls

coney-barrett_cnn_202009.json  python-duckduckgo  sample_data


In [ ]:
!ls python-duckduckgo

duckduckgo.py  LICENSE	MANIFEST  README.rst  setup.py


In [ ]:
%cd python-duckduckgo/

/content/python-duckduckgo


In [ ]:
!python setup.py install

Traceback (most recent call last):
  File "setup.py", line 2, in <module>
    from duckduckgo import __version__
  File "/content/python-duckduckgo/duckduckgo.py", line 134
    print "Invalid disambiguation number."
                                         ^
SyntaxError: Missing parentheses in call to 'print'. Did you mean print("Invalid disambiguation number.")?


### Duck Duck Go Python Library DuckDuckPy

Reference:

* https://github.com/ivankliuk/duckduckpy

In [ ]:
!pip install duckduckpy

  Created wheel for duckduckpy: filename=duckduckpy-0.2-cp36-none-any.whl size=9947 sha256=d8bcaf1c77922e31337bf898fa2ebea0be184e40d730d166192292bee961c4b5
  Stored in directory: /root/.cache/pip/wheels/0c/07/e3/086e992f0eb213ffb8ec53bfb8450a1da67343e050edc72327
Successfully built duckduckpy


In [ ]:
from duckduckpy import query

DuckDuckPy query API syntax

```
query(query_string, secure=False, container=u'namedtuple', verbose=False,
      user_agent=u'duckduckpy 0.2', no_redirect=False, no_html=False,
      skip_disambig=False)
```

In [ ]:
response = query("brett kavanaugh August 2018", container=u'dict', no_html=True)

In [ ]:
pprint.pprint(response)

{'abstract': '',
 'abstract_source': '',
 'abstract_text': '',
 'abstract_url': '',
 'answer': '',
 'answer_type': '',
 'definition': '',
 'definition_source': '',
 'definition_url': '',
 'entity': '',
 'heading': '',
 'image': '',
 'image_height': '',
 'image_is_logo': '',
 'image_width': '',
 'infobox': '',
 'meta': None,
 'redirect': '',
 'related_topics': [],
 'results': [],
 'type': ''}


In [ ]:
# response = query('coney barrett', container='dict') # namedtuple is used as a container
response = query('brett kavanaugh', container='dict') # namedtuple is used as a container

print(response)

{'image_height': 270, 'definition': '', 'definition_source': '', 'type': 'A', 'entity': 'infobox', 'image': '/i/3fbed2ef.jpg', 'answer': '', 'image_is_logo': 0, 'answer_type': '', 'image_width': 222, 'abstract_source': 'Wikipedia', 'definition_url': '', 'abstract_url': 'https://en.wikipedia.org/wiki/Brett_Kavanaugh', 'abstract_text': "Brett Michael Kavanaugh is an Associate Justice of the Supreme Court of the United States. He was nominated by President Donald Trump on July 9, 2018, and has served since October 6, 2018. He was previously a United States Circuit Judge of the United States Court of Appeals for the District of Columbia Circuit and worked as a staff lawyer for various offices of the federal government. Kavanaugh studied history at Yale University, where he joined Delta Kappa Epsilon fraternity. He then attended Yale Law School, after which he began his career as a law clerk and then a postgraduate fellow working under Judge Ken Starr. After Starr left the D.C. Circuit to b

In [ ]:
pprint.pprint(response)

{'abstract': 'Brett Michael Kavanaugh is an Associate Justice of the Supreme '
             'Court of the United States. He was nominated by President Donald '
             'Trump on July 9, 2018, and has served since October 6, 2018. He '
             'was previously a United States Circuit Judge of the United '
             'States Court of Appeals for the District of Columbia Circuit and '
             'worked as a staff lawyer for various offices of the federal '
             'government. Kavanaugh studied history at Yale University, where '
             'he joined Delta Kappa Epsilon fraternity. He then attended Yale '
             'Law School, after which he began his career as a law clerk and '
             'then a postgraduate fellow working under Judge Ken Starr. After '
             'Starr left the D.C. Circuit to become head of the Office of '
             'Independent Counsel, Kavanaugh assisted him with various '
             'investigations concerning President Bill Clint

In [ ]:
pprint.pprint(response.related_topics[0])

AttributeError: ignored

In [ ]:
!pip install gsearch

In [ ]:
from newsfetch.google import google_search
>>> google = google_search('Alcoholics Anonymous', 'https://timesofindia.indiatimes.com/')

Cloning into 'news-fetch'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 95 (delta 16), reused 0 (delta 0), pack-reused 65
Unpacking objects: 100% (95/95), done.


In [ ]:
!ls

news-fetch  sample_data


In [ ]:
!ls news-fetch

CODE_OF_CONDUCT.md  LICENSE    README.md	 setup.cfg
_config.yml	    newsfetch  requirements.txt  setup.py


In [ ]:
%cd news-fetch

/content/news-fetch


In [ ]:
!pip install -r requirements.txt

     |████████████████████████████████| 911kB 5.2MB/s 
     |████████████████████████████████| 22.3MB 1.5MB/s 
     |████████████████████████████████| 245kB 41.5MB/s 
     |████████████████████████████████| 92kB 9.4MB/s 
ERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.


In [ ]:
from newsfetch.google import google_search

ModuleNotFoundError: ignored

In [ ]:
from newsfetch.google import google_search
>>> google = google_search('Alcoholics Anonymous', 'https://timesofindia.indiatimes.com/')

In [ ]:
!pip install newspaper3k

     |████████████████████████████████| 215kB 8.1MB/s 
     |████████████████████████████████| 7.4MB 9.3MB/s 
     |████████████████████████████████| 92kB 13.4MB/s 
     |████████████████████████████████| 81kB 12.4MB/s 
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-cp36-none-any.whl size=13538 sha256=bfd420877823f9e0471f714ff658e6b533d7a4778ad23b17c38a58e54e4300e5
  Stored in directory: /root/.cache/pip/wheels/81/2b/43/a02ede72324dd40cdd7ca53aad718c7710628e91b8b0dc0f02
  Created wheel for jieba3k: filename=jieba3k-0.35.1-cp36-none-any.whl size=7398405 sha256=0fb2a4b5de62fa95efde3323bfb6f3dca818e289579416dbc4a09bfa584493d1
  Stored in directory: /root/.cache/pip/wheels/83/15/9c/a3f1f67e7f7181170ad37d32e503c35da20627c013f438ed34
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-cp36-none-any.whl size=3356 sha256=092f6168644d7db6efa36216294b1fecdf7a53ea7592f7434034be93bd7fd2fa
  Stored in directory: /root/.cache/pip/wheels/de/03/ca/778e3a7a627e3d98836cc890e7cb40c

In [ ]:
from newspaper import Article

url = 'http://fox13now.com/2013/12/30/new-year-new-laws-obamacare-pot-guns-and-drones/'
article = Article(url)

In [ ]:
article.download()
article.html

'<!DOCTYPE html>\n<html class="Page-body ArticlePage" lang="en" itemscope itemtype="http://schema.org/WebPage">\n<head>\n    <meta charset="UTF-8">\n\n    <link rel="stylesheet" href="https://ewscripps.brightspotcdn.com/resource/00000176-4e7c-d1e8-a3f7-eefe6ddf0000/styleguide/All.min.d947d3aa6bfaab6e7941349b2e19d4cf.gz.css">\n\n    <style>.newsletter-opt{box-sizing: border-box; padding: 20px;box-shadow: 1px 0px 9px #e7e7e7;border: 1px solid #e7e7e7;border-top:4px solid #318CC7;border-radius: 4px;margin: 30px 0;}\r\n.newsletter-opt #email{width: 80%;padding: 10px;margin: 8px 0;margin-right: 1%;border: 1px solid #ccc;border-radius: 4px;box-sizing: border-box;font-size: 1em;}\r\n.newsletter-opt #newSub{width: 18%;background-color: #005687;color: white;padding: 10px;margin: 8px 0;border: 1px solid #005687;border-radius: 4px;cursor: pointer;font-size: 1em;-webkit-appearance: none;}\r\n.newsletter-opt #emailForm{margin-bottom: 0;}\r\n.newsletter-opt span{font-weight:bold;color:#005687;}\r\n.

In [ ]:
article.parse()

In [ ]:
article.authors

[]

In [ ]:
article.publish_date

datetime.datetime(2013, 12, 30, 0, 0)

In [ ]:
article.text

'By Leigh Ann Caldwell\n\nWASHINGTON (CNN) — Not everyone subscribes to a New Year’s resolution, but Americans will be required to follow new laws in 2014.\n\nSome 40,000 measures taking effect range from sweeping, national mandates under Obamacare to marijuana legalization in Colorado, drone prohibition in Illinois and transgender protections in California.\n\nAlthough many new laws are controversial, they made it through legislatures, public referendum or city councils and represent the shifting composition of American beliefs.\n\nFederal: Health care, of course, and vending machines\n\nThe biggest and most politically charged change comes at the federal level with the imposition of a new fee for those adults without health insurance.\n\nFor 2014, the penalty is either $95 per adult or 1% of family income, whichever results in a larger fine.\n\nThe Obamacare, of Affordable Care Act, mandate also requires that insurers cover immunizations and some preventive care.\n\nAdditionally, mil

In [ ]:
article.top_image

'https://ewscripps.brightspotcdn.com/dims4/default/d49dab0/2147483647/strip/true/crop/400x210+0+8/resize/1200x630!/quality/90/?url=http%3A%2F%2Fmediaassets.fox13now.com%2Ftribune-network%2Ftribkstu-files-wordpress%2F2012%2F04%2Fnational-news-e1486938949489.jpg'

In [ ]:
article.movies

[]

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
article.nlp()

In [ ]:
article.keywords

['minimum',
 'states',
 'drones',
 'family',
 'wage',
 'obamacare',
 'latest',
 'law',
 'guns',
 'leave',
 'state',
 'pot',
 'laws',
 'national']

In [ ]:
article.summary

'Oregon: Family leave in Oregon has been expanded to allow eligible employees two weeks of paid leave to handle the death of a family member.\nArkansas: The state becomes the latest state requiring voters show a picture ID at the voting booth.\nMinimum wage and former felon employmentWorkers in 13 states and four cities will see increases to the minimum wage.\nNew Jersey residents voted to raise the state’s minimum wage by $1 to $8.25 per hour.\nCalifornia is also raising its minimum wage to $9 per hour, but workers must wait until July to see the addition.'

## **Scrape CNN News**

In [ ]:
import newspaper

In [ ]:
cnn_paper = newspaper.build('http://cnn.com')

In [ ]:
for article in cnn_paper.articles:
  print(article.url)

http://cnn.com/business/media
http://cnn.com/travel/news
http://cnn.com/2020/12/13/asia/peter-humphrey-china-prisoners-intl-hnk/index.html
http://cnn.com/2020/12/04/economy/china-digital-yuan-currency-intl-hnk/index.html
http://cnn.com/2020/12/07/china/south-china-sea-bases-military-intl-hnk/index.html
http://cnn.com/2020/12/03/tech/autox-robotaxi-china-intl-hnk/index.html
http://cnn.com/2020/12/03/asia/hong-kong-democracy-arrests-protest-intl-hnk/index.html
http://www.cnn.com/interactive/2016/12/specials/vanishing/
http://cnn.com/2016/12/11/world/vanishing-mckenzie-botswana-elephants/index.html
http://cnn.com/2016/12/08/world/sutter-giraffe-extinction/index.html
http://cnn.com/2016/12/11/world/vanishing-sutter-amphibian-extinction/index.html
http://www.cnn.com/interactive/2016/12/world/midway-plastic-island/
http://cnn.com/2016/12/11/world/vanishing-sutter-coral-madagascar/index.html
http://cnn.com/2016/12/11/us/vanishing-sutter-franklins-bumblebee/index.html
http://cnn.com/2016/10/27

In [ ]:
for category in cnn_paper.category_urls():
  print(category)

http://cnn.com
http://cnn.com/china
http://cnn.com/more
http://cnn.com/travel
http://cnn.com/audio
http://us.cnn.com
http://cnn.com/europe
http://cnn.com/business
http://cnn.com/videos
https://money.cnn.com
http://cnnespanol.cnn.com
http://cnn.com/health
http://cnn.com/americas
http://cnn.com/middle-east
http://cnn.com/india
http://cnn.com/uk
http://arabic.cnn.com
http://cnn.com/entertainment
http://edition.cnn.com
http://cnn.com/transcripts
http://cnn.com/politics
http://cnn.com/vr
http://cnn.com/accessibility
http://cnn.com/asia
http://cnn.it
http://cnn.com/australia
http://cnn.com/africa
http://cnn.com/style
http://cnn.com/opinions
http://cnn.com/weather
https://www.cnn.com
http://cnn.com/world
http://cnn.com/collection
http://cnn.com/us


In [ ]:
cnn_article = cnn_paper.articles[0]
cnn_article.download()
cnn_article.parse()
cnn_article.nlp()

In [ ]:
cnn_article.authors

[]

In [ ]:
cnn_article.keywords

['business', 'headlines', 'latest', 'cnn']

In [ ]:
import requests

In [ ]:
from newspaper import fulltext

url = 'http://cnn.com/2020/10/15/africa/nigerians-protest-police-brutality-intl/index.html'
html = requests.get(url).text
text = fulltext(html)

In [ ]:
text

'(CNN) Nigerian protesters demanding an end to police brutality returned to the streets on Wednesday, saying they were unconvinced by the creation of a new police unit and a pledge not to use violence against demonstrators.\n\nProtesters have staged daily marches nationwide for a week, calling for an overhaul of police forces. Police have responded to the demonstrations with beatings, tear gas and gunfire, which human rights group Amnesty International said had killed at least 10 people.\n\nThe protests have prompted a raft of announcements. The Special Anti-Robbery Squad (SARS), a police unit that demonstrators have long accused of beatings, killings and extortion, was officially disbanded on Sunday.\n\nOn Tuesday, police agreed to stop using force against protesters. They also announced the formation of a new unit, the Special Weapons and Tactics team (SWAT), to "fill the gaps" left by the disbanded SARS.\n\nBut protesters said on Wednesday they feared the new unit will simply be a r

In [ ]:
from newspaper import Article
url = 'http://www.bbc.co.uk/zhongwen/simp/chinese_news/2012/12/121210_hongkong_politics.shtml'

a = Article(url, language='zh') # Chinese

a.download()
a.parse()


Building prefix dict from /usr/local/lib/python3.6/dist-packages/jieba/dict.txt ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.3415873050689697 seconds.
Prefix dict has been built succesfully.


In [ ]:
print(a.text[:150])

香港行政长官梁振英在各方压力下就其大宅的违章建筑（僭建）问题到立法会接受质询，并向香港民众道歉。

梁振英此前承认早在去年参选行政长官之前就已知悉其住宅的违建问题，引发诚信危机。

梁振英在星期二（12月10日）的答问大会开始之际在其演说中道歉，但强调他在违章建筑问题上没有隐瞒的意图和动机。

不过
